# Получение данных по фьючерсам через Alor OpenAPI. Проверка на коинтеграцию.

In [1]:
import os
import json
from datetime import datetime
from io import StringIO

import numpy as np
import pandas as pd
import requests

import matplotlib.pyplot as plt
import seaborn as sns

## Скачивание данных через API Алор

In [62]:
def fetch_fut_data(fut_code: str) -> pd.DataFrame:
    """Функция выкачивает данные по shortcode фьючерса.
    
    Длительность таймфрейма 'tf'. В качестве значения можно указать точное количество секунд или код таймфрейма:
    15 — 15 секунд
    60 — 60 секунд или 1 минута
    3600 — 3600 секунд или 1 час
    D — сутки (соответствует значению 86400)
    W — неделя (соответствует значению 604800)
    M — месяц (соответствует значению 2592000)
    Y — год (соответствует значению 31536000)
    
    """
    
    url = "https://api.alor.ru/md/v2/history"
    
    payload = {}
    headers = {
      'Accept': 'application/json',
      'Authorization': 'Bearer <token>'
    }
    
    from_time = int(datetime(2024, 1, 1, 0, 0).timestamp())
    to_time = int(datetime(2025, 12, 31, 0, 0).timestamp())
    
    get_params = {
        'symbol': fut_code,
        'exchange': 'MOEX',
        'instrumentGroup': 'RFUD',
        'tf': 3600,
        'from': from_time,  # 1735653600,
        'to': to_time,      # 1740751200,
        'splitAdjust': 'true',
        'format': 'simple',
        'jsonResponse': 'true',
    }
    
    response = requests.request("GET", url, headers=headers, data=payload, params=get_params)
    # data = json.loads(response.text)
    data = StringIO(response.text)
    
    return pd.json_normalize(pd.read_json(data)['history'])

In [93]:
def fetch_all_current_futures_data() -> pd.DataFrame:
    url = "https://api.alor.ru/md/v2/Securities?sector=FORTS&exchange=MOEX&instrumentGroup=RFUD&limit=50"
    
    payload = {}
    headers = {
      'Accept': 'application/json',
      # 'Authorization': 'Bearer <token>'
    }
    
    response = requests.request("GET", url, headers=headers, data=payload)
    return pd.json_normalize(json.loads(response.text))
    

In [94]:
def save_fut_data(short_code: str, fut_data: pd.DataFrame):
    
    dir_path = f"./data/{short_code}/"
    filename = short_code + ".csv"
    
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)
        
    fut_data.to_csv(dir_path + filename)

In [95]:
def save_futures_data(futures_codes: pd.Series):
    
    for code in futures_codes:
        print(f"Запрос {code}")
        _df = fetch_fut_data(code)
        print(_df.head(1))
        save_fut_data(code, _df)

    print("DONE.")
    

In [96]:
futures = fetch_all_current_futures_data()

In [97]:
futures.head()

,symbol,shortname,description,exchange,market,type,lotsize,facevalue,cfiCode,cancellation,...,currency,ISIN,yield,board,primary_board,tradingStatus,tradingStatusInfo,complexProductCategory,priceMultiplier,priceShownUnits
0,CNY-3.25,CRH5,CRH5,MOEX,FORTS,Фьючерсный контракт CNY-3.25,1,1000,FFXCSX,2025-03-20T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1
1,CNY-6.25,CRM5,CRM5,MOEX,FORTS,Фьючерсный контракт CNY-6.25,1,1000,FFXCSX,2025-06-19T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
2,CNYRUBF,CNYRUBF,CNYRUBF,MOEX,FORTS,CNYRUBF,1,1000,FFCCSX,2100-01-01T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
3,CNY-3.25-6.25,CRH5CRM5,CRH5CRM5,MOEX,FORTS,Календарный спред CNY-3.25-6.25,1,1000,FMXXSX,2025-03-20T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1
4,NG-3.25,NGH5,NGH5,MOEX,FORTS,Фьючерсный контракт NG-3.25,1,100,FCXCSX,2025-03-27T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1


In [98]:
futures.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 31 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   symbol                  50 non-null     object 
 1   shortname               50 non-null     object 
 2   description             50 non-null     object 
 3   exchange                50 non-null     object 
 4   market                  50 non-null     object 
 5   type                    50 non-null     object 
 6   lotsize                 50 non-null     int64  
 7   facevalue               50 non-null     int64  
 8   cfiCode                 50 non-null     object 
 9   cancellation            50 non-null     object 
 10  minstep                 50 non-null     float64
 11  rating                  50 non-null     int64  
 12  marginbuy               50 non-null     float64
 13  marginsell              50 non-null     float64
 14  marginrate              50 non-null     floa

In [99]:
futures.to_csv('futures50.csv')

In [ ]:
for idx, code in enumerate(futures['shortname']):
    print(idx, f". {code}...", end='')
    _df = fetch_fut_data(code)
    # print(_df.head(1))
    save_fut_data(code, _df)
    print("ok")

## Препроцессинг / склейка данных

**Обозначение месяца/квартала:**
- F — январь (January)
- G — февраль (February)
- **H — март (March)**
- J — апрель (April)
- K — май (May)
- **M — июнь (June)**
- N — июль (July)
- Q — август (August)
- **U — сентябрь (September)**
- V — октябрь (October)
- X — ноябрь (November)
- **Z — декабрь (December)**

Фьючерсы CHMF-3.25 на акции Северсталь. 
("CHM5", "RFUD", "CHMF-6.25")

GMKN-3.25 - Нор. никель. GKH5. GK

Фьючерсы мосбиржи:
- https://www.moex.com/ru/derivatives/equity/stocks/

Флор API
- https://alor.dev/docs/api/http/md-v-2-history-get

In [69]:
from pathlib import Path

def find_fut_csv(quart_code: str = ''):
    base_dir = Path("data")
    pattern = "*.csv" if not quart_code else f"*{quart_code}.csv"
    
    # Рекурсивный поиск всех файлов, соответствующих шаблону
    csv_paths = base_dir.rglob(pattern)
    csv_paths = list(map(str, filter(lambda path: ".ipynb" not in str(path), csv_paths)))
    
    return csv_paths

In [106]:
def merge_fut_data(fut_files: list = []):

    if not fut_files:
        fut_files = find_fut_csv()

    merged_df = None
    
    for idx, file in enumerate(fut_files):
        if idx == 0:
            merged_df = pd.read_csv(file, index_col=0).loc[:, ['time', 'close']]
            merged_df.rename(columns={"close": file.split('/')[1].lower()}, inplace=True)
            continue
            
        _df = pd.read_csv(file, index_col=0).loc[:, ['time', 'close']]
        _df.rename(columns={"close": file.split('/')[1].lower()}, inplace=True)
        
        merged_df = pd.merge(merged_df, _df, on='time', how='inner').sort_values(by='time').set_index('time')
        # print()
        # print(idx, file)
        # print(merged_df.head(5))
        
    return merged_df

In [109]:
df = merge_fut_data(find_fut_csv('M5'))
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 747 entries, 1734440400 to 1742464800
Data columns (total 22 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   gdm5      747 non-null    float64
 1   ssm5      747 non-null    float64
 2   mxm5      747 non-null    float64
 3   gzh5gzm5  747 non-null    float64
 4   sim5      747 non-null    float64
 5   eum5      747 non-null    float64
 6   svh5svm5  747 non-null    float64
 7   svm5      747 non-null    float64
 8   rim5      747 non-null    float64
 9   glm5      747 non-null    float64
 10  edh5edm5  747 non-null    float64
 11  gkm5      747 non-null    float64
 12  crm5      747 non-null    float64
 13  mmm5      747 non-null    float64
 14  gzm5      747 non-null    float64
 15  sih5sim5  747 non-null    float64
 16  crh5crm5  747 non-null    float64
 17  edm5      747 non-null    float64
 18  srm5      747 non-null    float64
 19  vbm5      747 non-null    float64
 20  nam5      747 non-nul

In [110]:
df.describe()

,gdm5,ssm5,mxm5,gzh5gzm5,sim5,eum5,svh5svm5,svm5,rim5,glm5,...,crm5,mmm5,gzm5,sih5sim5,crh5crm5,edm5,srm5,vbm5,nam5,vkm5
count,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,...,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000,747.000000
mean,2913.987149,1387.955823,326207.061580,788.801874,99174.808568,102330.625167,0.917363,33.234217,104669.116466,9221.224498,...,13.558436,3258.967269,16408.409639,3324.136546,0.549353,1.036397,31509.625167,9313.257028,21329.647925,3515.448461
std,90.251669,178.304998,18118.435795,131.980829,6735.330565,5082.477565,0.108051,0.932956,12033.022047,371.462025,...,0.940504,181.553765,1867.739552,887.003828,0.063934,0.023153,2111.624370,613.824990,781.038156,171.353541
min,2713.200000,876.000000,264000.000000,553.000000,85552.000000,92832.000000,0.310000,31.010000,80250.000000,8418.000000,...,11.713000,2668.600000,12052.000000,1134.000000,0.288000,1.001100,24853.000000,7178.000000,19564.000000,2620.000000
25%,2842.300000,1370.500000,313000.000000,673.000000,93409.000000,97592.500000,0.890000,32.480000,93920.000000,8869.500000,...,12.744500,3121.150000,14849.000000,2803.500000,0.552000,1.021150,29536.000000,8843.500000,20858.000000,3408.500000
50%,2947.100000,1429.000000,328200.000000,722.000000,98362.000000,101406.000000,0.940000,33.320000,108400.000000,9285.000000,...,13.308000,3281.150000,17116.000000,3512.000000,0.570000,1.029900,32244.000000,9471.000000,21511.000000,3524.000000
75%,2982.000000,1521.000000,341437.500000,917.000000,105675.000000,107712.000000,0.980000,33.930000,115785.000000,9580.950000,...,14.486500,3413.200000,18228.000000,3845.500000,0.582500,1.039400,33498.500000,9759.000000,21908.000000,3623.500000
max,3068.800000,1656.000000,356600.000000,1045.000000,108640.000000,111041.000000,1.150000,35.120000,125200.000000,9769.300000,...,14.970000,3562.200000,19050.000000,6275.000000,0.638000,1.086300,34695.000000,10499.000000,22567.000000,3895.000000


In [24]:
# from ydata_profiling import ProfileReport

In [25]:
# profile = ProfileReport(df, title="Profiling Report", explorative=True)

In [26]:
# profile.to_file('report.html')

In [49]:
# corr_matrix = df.corr()

In [ ]:
# plt.figure(figsize=(12, 6))
# sns.heatmap(df.corr(), annot=False, cmap='coolwarm', fmt=".2f")
# plt.title("Тепловая карта корреляций")
# plt.show()

In [ ]:
# threshold = 0.9
# high_corr_pairs = []
# # Проход по матрице корреляций
# for i in range(len(corr_matrix.columns)):
#     for j in range(i + 1, len(corr_matrix.columns)):  # Избегаем дублирования и диагонали
#         if abs(corr_matrix.iloc[i, j]) > threshold:  # Учитываем абсолютное значение корреляции
#             col1 = corr_matrix.columns[i]
#             col2 = corr_matrix.columns[j]
#             corr_value = corr_matrix.iloc[i, j]
#             high_corr_pairs.append((col1, col2, corr_value))

# # Вывод результата
# print("Пары с корреляцией выше", threshold, ":")
# for pair in high_corr_pairs:
#     print(f"{pair[0]} и {pair[1]}: {pair[2]:.2f}")

In [73]:
from statsmodels.tsa.stattools import coint

# Функция для проверки коинтеграции между двумя рядами
def check_cointegration(series1, series2):
    # Выполняем тест Engle-Granger
    score, p_value, _ = coint(series1, series2)
    return p_value  # Возвращаем p-value

In [111]:
import itertools

# Список колонок
columns = df.columns

# Словарь для хранения результатов
coint_results = {}

# Перебор всех возможных пар
for col1, col2 in itertools.combinations(columns, 2):
    p_value = check_cointegration(df[col1], df[col2])
    coint_results[(col1, col2)] = p_value

# Вывод результатов
for pair, p_value in coint_results.items():
    if p_value < 0.05:
        print(f"Пара {pair[0]} / {pair[1]}:\tp-value = {round(p_value, 4)}")

Пара ssm5 / gzh5gzm5:	p-value = 0.0444
Пара ssm5 / edh5edm5:	p-value = 0.0367
Пара mxm5 / sim5:	p-value = 0.014
Пара mxm5 / eum5:	p-value = 0.0043
Пара mxm5 / rim5:	p-value = 0.046
Пара mxm5 / glm5:	p-value = 0.0168
Пара mxm5 / edh5edm5:	p-value = 0.0018
Пара mxm5 / crm5:	p-value = 0.0235
Пара mxm5 / gzm5:	p-value = 0.0011
Пара mxm5 / edm5:	p-value = 0.0026
Пара mxm5 / nam5:	p-value = 0.0399
Пара sim5 / rim5:	p-value = 0.0282
Пара sim5 / srm5:	p-value = 0.0022
Пара eum5 / rim5:	p-value = 0.0176
Пара eum5 / srm5:	p-value = 0.0075
Пара svh5svm5 / svm5:	p-value = 0.0043
Пара svh5svm5 / rim5:	p-value = 0.0079
Пара svh5svm5 / glm5:	p-value = 0.0059
Пара svh5svm5 / edh5edm5:	p-value = 0.0058
Пара svh5svm5 / gkm5:	p-value = 0.0111
Пара svh5svm5 / crm5:	p-value = 0.0063
Пара svh5svm5 / mmm5:	p-value = 0.0107
Пара svh5svm5 / gzm5:	p-value = 0.0096
Пара svh5svm5 / sih5sim5:	p-value = 0.0198
Пара svh5svm5 / crh5crm5:	p-value = 0.0079
Пара svh5svm5 / edm5:	p-value = 0.0022
Пара svh5svm5 / srm5:	p-

In [113]:
futures.loc[(futures.shortname == 'GKM5') | (futures.shortname == 'GZM5') , ['symbol', 'type'] ]

,symbol,type
26,GMKN-6.25,Фьючерсный контракт GMKN-6.25
32,GAZR-6.25,Фьючерсный контракт GAZR-6.25


In [79]:
futures.head()

,symbol,shortname,description,exchange,market,type,lotsize,facevalue,cfiCode,cancellation,...,currency,ISIN,yield,board,primary_board,tradingStatus,tradingStatusInfo,complexProductCategory,priceMultiplier,priceShownUnits
0,CNY-3.25,CRH5,CRH5,MOEX,FORTS,Фьючерсный контракт CNY-3.25,1,1000,FFXCSX,2025-03-20T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1
1,CNY-6.25,CRM5,CRM5,MOEX,FORTS,Фьючерсный контракт CNY-6.25,1,1000,FFXCSX,2025-06-19T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
2,CNYRUBF,CNYRUBF,CNYRUBF,MOEX,FORTS,CNYRUBF,1,1000,FFCCSX,2100-01-01T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
3,CNY-3.25-6.25,CRH5CRM5,CRH5CRM5,MOEX,FORTS,Календарный спред CNY-3.25-6.25,1,1000,FMXXSX,2025-03-20T00:00:00.0000000,...,RUB,None,None,RFUD,RFUD,18,нет торгов или торги закрыты,2,1,1
4,NG-3.25,NGH5,NGH5,MOEX,FORTS,Фьючерсный контракт NG-3.25,1,100,FCXCSX,2025-03-27T00:00:00.0000000,...,USD,None,None,RFUD,RFUD,2,перерыв в торгах,2,1,1
